[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/05-agents-intro.ipynb)

#### LangChain Essentials Course

# LangChain Agents Intro

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Agents, adding the ability to use tools such as search and calculators to complete tasks that normal LLMs cannot fufil. In this example we will be using Google Gemini 2.5 Flash as it's free.

---

> ⚠️ If using LangSmith, add your API key below:

In [ ]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-agents-intro-openai"

---

## Introduction to Tools

Tools are a way augment our LLMs with code execution. A tool is simply a function formatted so that our agent can undertstand how to use it, and then execute it. Let's start by creating a few simple tools.

We can use the `@tool` decorator to create an LLM-compatible tool from a standard python function — this function should include a few things for optimal performance:

* A docstring describing what the tool does and when it should be used, this will be read by our LLM/agent and used to decide when to use the tool, and also how to use the tool.

* Clear parameter names that ideally tell the LLM what each parameter is, if it isn't clear we make sure the docstring explains what the parameter is for and how to use it.

* Both parameter and return type annotations.

In [3]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

With the `@tool` decorator our function is turned into a `StructuredTool` object, which we can see below:

In [4]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x000002DAD949E3E0>)

We can see the tool name, description, and arg schema:

In [5]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [6]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [7]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the below:

In [8]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

This is then passed into the tool function as `kwargs` (keyword arguments) as indicated by the `**` operator - the `**` operator is used to unpack the dictionary into keyword arguments.

In [9]:
exponentiate.func(**llm_output_dict)

25

This covers the basics of tools and how they work, let's move on to creating the agent itself.

## Creating an Agent

We're going to construct a simple tool calling agent. We will use **L**ang**C**hain **E**pression **L**anguage (LCEL) to construct the agent. We will cover LCEL more in the next chapter, but for now - all we need to know is that our agent will be constructed using syntax and components like so:


```
agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)
```

We need this agent to remember previous interactions within the conversation. To do that, we will use the `ChatPromptTemplate` with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent will write it's _"notes"_ as it is working through multiple internal thought and tool-use steps to produce a final output to the user.

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

Next, we must define our LLM, we will use the `gemini-2.5-flash` model with a `temperature` of `0.0`.

In [ ]:
import os
from getpass import getpass
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = ""

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
)

c:\Users\micro\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


When creating an agent we need to add conversational memory to make the agent remember previous interactions. We'll be using the older `ConversationBufferMemory` class rather than the newer `RunnableWithMessageHistory` — the reason being that we will also be using the older `create_tool_calling_agent` and `AgentExecutor` method and class.

In the `05` chapter we will be using the newer `RunnableWithMessageHistory` class as we'll be building a custom `AgentExecutor`.

In [12]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # must align with MessagesPlaceholder variable_name
    return_messages=True  # to return Message objects
)

C:\Users\micro\AppData\Local\Temp\ipykernel_20592\287820297.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Now we will initialize our agent. For that we need:

* `llm`: as already defined
* `tools`: to be defined (just a list of our previously defined tools)
* `prompt`: as already defined
* `memory`: as already defined

In [13]:
from langchain.agents import create_tool_calling_agent

tools = [add, subtract, multiply, exponentiate]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

Our `agent` by itself is like one-step of our agent execution loop. So, if we call the `agent.invoke` method it will get the LLM to generate a single response and go no further, so no tools will be executed, and no next iterations will be performed.

We can see this by asking a query that should trigger a tool call:

In [14]:
agent.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps": []  # agent will append it's internal steps here
})

[ToolAgentAction(tool='multiply', tool_input={'y': 7.68, 'x': 10.7}, log="\nInvoking: `multiply` with `{'y': 7.68, 'x': 10.7}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"y": 7.68, "x": 10.7}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--35c1c073-226f-4986-8fb7-dec0e6ffb914-0', tool_calls=[{'name': 'multiply', 'args': {'y': 7.68, 'x': 10.7}, 'id': '845cd9b8-264f-41d2-9861-5834f94debbb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 222, 'output_tokens': 24, 'total_tokens': 329, 'input_token_details': {'cache_read': 0}})], tool_call_id='845cd9b8-264f-41d2-9861-5834f94debbb')]

Here, we can see the LLM has generated that we should use the `multiply` tool and the tool input should be `{"x": 10.7, "y": 7.68}`. However, the tool is not executed. For that to happen we need an agent execution loop, which will handle the multiple iterations of generation to tool calling to generation, etc.

We use the `AgentExecutor` class to handle the execution loop:

In [15]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

Do note that a specific version of langchain-google-genai will be required for the code below to work. The version is 2.0.7, use `pip install langchain-google-genai==2.0.7`.

Now let's try the same query with the executor, note that the `intermediate_steps` parameter that we added before is no longer needed as the executor handles it internally.

In [16]:
agent_executor.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'y': 7.68, 'x': 10.7}`


82.1759999999999910.7 multiplied by 7.68 is 82.176.

> Finished chain.


{'input': 'what is 10.7 multiplied by 7.68?',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={})],
 'output': '10.7 multiplied by 7.68 is 82.176.'}

We can see that the `multiply` tool was invoked, producing the observation of `82.175999...`. After the observation was provided, we can see that the LLM then generated a final response of:

```
10.7 multiplied by 7.68 is approximately 82.18.
```

This final response was generated based on the original query and the tool output (ie the _observation_). We can also confirm that this answer is accurate:

In [17]:
10.7*7.68

82.17599999999999

Let's test our agent with some memory and tool use. First, we tell it our name, then we will perform a few tool calls, then see if the agent can still recall our name.

First, give the agent our name:

In [23]:
agent_executor.invoke({
    "input": "My name is James",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Hello James! How can I help you today?

> Finished chain.


{'input': 'My name is James',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello James! How can I help you today?\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The answer is 1331.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The answer is 1331.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm sorry, I don't

Now let's try and get the agent to perform multiple tool calls within a single execution loop:

In [24]:
agent_executor.invoke({
    "input": "What is nine plus 10, minus 4 * 2, to the power of 3",
    "chat_history": memory
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'y': 2.0, 'x': 4.0}`


8.0
Invoking: `add` with `{'y': 10.0, 'x': 9.0}`


19.0
Invoking: `subtract` with `{'y': 19.0, 'x': 8.0}`


11.0
Invoking: `exponentiate` with `{'y': 3.0, 'x': 11.0}`


1331.0The answer is 1331.

> Finished chain.


{'input': 'What is nine plus 10, minus 4 * 2, to the power of 3',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello James! How can I help you today?\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The answer is 1331.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The answer is 1331.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  A

Let's confirm that the answer is accurate:

In [25]:
9+10-(4*2)**3

-493

Perfect, now let's see if the agent can still recall our name:

In [26]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Your name is James.


> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello James! How can I help you today?\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The answer is 1331.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The answer is 1331.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm sorry, I don't 

The agent has successfully recalled our name. Let's move on to another agent example.

## SerpAPI Weather Agent

In this example, we'll be using the same agent and executor setup as before, but we'll be adding the [SerpAPI](https://serpapi.com/users/sign_in) service to allow our agent to search the web for information.

To use this tool, you need an API key, with the free plan you can use up to 100 searches per month.

In [ ]:
os.environ["SERPAPI_API_KEY"] = ""

Here we will load the `serpapi` tool directly from the prebuilt tools that LangChain provides.

In [54]:
from langchain.agents import load_tools

toolbox = load_tools(tool_names=['serpapi'], llm=llm)

These custom tools can look into your IP address, find out where you are currently, then we will also use a secondary function to get the current date and time, then we will use this information to feed into the SerpAPI to find us the weather pattern in your area and at the time of the function calling.

In [55]:
import requests
from datetime import datetime

@tool
def get_location_from_ip():
    """Get the geographical location based on the IP address."""
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        if 'loc' in data:
            latitude, longitude = data['loc'].split(',')
            data = (
                f"Latitude: {latitude},\n"
                f"Longitude: {longitude},\n"
                f"City: {data.get('city', 'N/A')},\n"
                f"Country: {data.get('country', 'N/A')}"
            )
            return data
        else:
            return "Location could not be determined."
    except Exception as e:
        return f"Error occurred: {e}"

@tool
def get_current_datetime() -> str:
    """Return the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

We can create our prompt, this time we'll skip the `chat_history` part as we don't need it. However, you can add it if preferred.

In [56]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

## ReAct Framework

So basically the previous code required the llm to output in a very specific format so that the action input can be properly sent, but this removed that complexity and made it simply, action and action input.

To be more specific:

The Previous Method (Structured Tool Calling)

The Goal: To get the LLM to act like a computer program. It was expected to generate a perfectly formatted, machine-readable JSON object.

The Output Format: The LLM had to produce something like this:

{
  "tool_name": "get_location_from_ip",
  "arguments": {}
}

The Point of Failure: This is a very strict contract. As you saw, the langchain-google-genai library and the AgentExecutor got out of sync. The executor was sending a response back to the model without the tool_name field, causing the Gemini API to reject it with the error function_response.name: Name cannot be empty. It's like trying to submit a form with a required field left blank—the system immediately rejects it.

The New Method (ReAct Framework)

The Goal: To get the LLM to reason like a human assistant that has access to tools. It's a conversation between the LLM's "thoughts" and the results of its actions.

The Output Format: The LLM only needs to produce simple, human-readable plain text:

Thought: I need to figure out where the user is. I can use a tool for that.
Action: get_location_from_ip
Action Input:

Why It's Robust:
Simplicity: It's much easier and more natural for a language model to generate these simple lines of text than it is to generate a perfectly structured JSON object. The risk of a formatting error is almost zero.
Removes Complexity: The AgentExecutor's job becomes much simpler. Instead of parsing complex JSON, it just needs to look for the keywords Action: and Action Input:. This simple text parsing is far less brittle and works consistently across different model versions.

Now we create our full `tools` list, our `agent`, and the `agent_executor`:

In [59]:
from langchain.agents import create_react_agent
from langchain import hub

# Your tools list remains exactly the same
tools = toolbox + [get_current_datetime, get_location_from_ip]

# 1. Get a prompt that is specifically designed for ReAct agents
# This pulls a well-tested, robust prompt from the LangChain Hub.
prompt = hub.pull("hwchase17/react-chat")

# 2. Create the ReAct agent
# This helper function combines the LLM, tools, and the ReAct prompt.
agent = create_react_agent(llm, tools, prompt)

# 3. Create the AgentExecutor
# This is the same as before, but it now runs our new ReAct agent.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

For me I have to specify to the AI as I live in the UK and alot of places in the UK also exist in USA, which is why I explicitly state to use the country in the search as well as the town / city.

In [61]:
out = agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time right now? "
        "How is the weather where I am? Please give me degrees in Celsius"
    ), 
    "chat_history": memory.chat_memory.messages
})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_current_datetime
Action Input:2025-09-26 01:50:57Do I need to use a tool? Yes
Action: get_location_from_ip
Action Input:Latitude: 1.3280,
Longitude: 103.7910,
City: Bukit Timah,
Country: SGDo I need to use a tool? Yes
Action: Search
Action Input: weather in Bukit Timah, SG in Celsius{'type': 'weather_result', 'temperature': '28', 'unit': 'Celsius', 'precipitation': '0%', 'humidity': '78%', 'wind': '8 km/h', 'location': 'Bukit Timah', 'date': 'Friday 1:00 AM', 'weather': 'Cloudy'}Do I need to use a tool? No
Final Answer: The current date and time is 2025-09-26 01:50:57. The weather in Bukit Timah, SG is cloudy with a temperature of 28 degrees Celsius, 0% precipitation, 78% humidity, and wind at 8 km/h.

> Finished chain.


In [62]:
from IPython.display import display, Markdown

display(Markdown(out["output"]))

The current date and time is 2025-09-26 01:50:57. The weather in Bukit Timah, SG is cloudy with a temperature of 28 degrees Celsius, 0% precipitation, 78% humidity, and wind at 8 km/h.

That's the correct answer, and we even get the approximate answer in Celsius despite the tool returning the temperature in Fahrenheit.

We've finished our into to LangChain Agents, in the next chapter we will be looking at how to create custom agents and executors.

---